In [ ]:
datatype = 'train'
low_base = 0
high_base = 9000

In [ ]:
!python -m spacy download en_core_web_sm
!pip install amrlib
!pip install Penman

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 66.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.2/354.2 kB 10.7 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - \ | / done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 kB 4.5 MB/s eta 0:00:00
  Created wheel for smatch: filename=smatch-1.0.4-py3-none-any.whl size=24055 sha256=15b9bf865aa8caa48fd37047e290f2447065575622caecdeafd1ea88e76da866
  Stored in directory: /root/.cache/pip/wheels/53/56/c6/da1826db6293069ea23fbff8f0a027939ee8087b0061281355
Successfully built smatch


In [ ]:
path_to_AMR3 = '**your path to AMR3.parsing.pt' #'/kaggle/input/amrpickledmodel/AMR3.parsing.pt'
!rm -r spring

rm: cannot remove 'spring': No such file or directory


In [ ]:
!git clone https://github.com/SapienzaNLP/spring.git

Cloning into 'spring'...
remote: Enumerating objects: 129, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 129 (delta 45), reused 44 (delta 27), pack-reused 61
Receiving objects: 100% (129/129), 504.09 KiB | 13.26 MiB/s, done.
Resolving deltas: 100% (60/60), done.


In [ ]:
!cd spring && pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 674.8/674.8 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 50.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 74.9 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=c8e9706d9ebc828e8a362374a21302a042c4d7a973b9e77634b735c642fbf728
  Stored in directory: /root/.cache/pip/wheels/5b/e0/77/05245143a5b31f65af6a21f7afd3219e9fa4896f918af45677
Successfully built sacremoses
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.2
    Uninstalling tokenizers-0.13.2:
      Successfully uninstalled tokenizers-0.13.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.27.4
    Uninstalling transformers-4.27.4:
      Successfully uninstalled transf

In [ ]:
!cd spring && pip install -e .

Obtaining file:///kaggle/working/spring
  Preparing metadata (setup.py) ... - done
  Running setup.py develop for spring-amr


In [ ]:
from pathlib import Path

import penman
import torch
from tqdm import tqdm
from spring.spring_amr import ROOT
from spring.bin.predict_amrs_from_plaintext import read_file_in_batches
from spring.spring_amr.evaluation import predict_amrs, compute_smatch
from spring.spring_amr.penman import encode
from spring.spring_amr.utils import instantiate_loader, instantiate_model_and_tokenizer

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [ ]:
with open('temp.txt', 'w') as f:
  f.write("Hi my name is John Doe I work for the AI tehcnology company, FNRH.")

In [ ]:
!ls

__notebook__.ipynb  spring  temp.txt


In [ ]:
args = {'texts': ['temp.txt'],
        'device': 'cuda',
        'model':'facebook/bart-large',
        'penman_linearization': True,
        'use_pointer_tokens': True,
        'raw_graph': False,
        'checkpoint': path_to_AMR3,
        'return_all': False,
        'use_recategorization': False,
        'batch_size': 1,
        'beam_size': 5}
args['only_ok'] = True
args['restore_name_ops'] = False

In [ ]:
if True:
    device = torch.device(args['device'])
    model, tokenizer = instantiate_model_and_tokenizer(
        args['model'],
        dropout=0.,
        attention_dropout=0,
        penman_linearization=args['penman_linearization'],
        use_pointer_tokens=args['use_pointer_tokens'],
    )
    model.load_state_dict(torch.load(args['checkpoint'], map_location='cpu')['model'])
    model.to(device)
    model.eval()

Downloading:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset
dataset = load_dataset("ecthr_cases")

Generating train split:   0%|          | 0/9000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset ecthr_cases downloaded and prepared to /root/.cache/huggingface/datasets/ecthr_cases/alleged-violation-prediction/1.1.0/8922a012792758e64921d4a66d42adf759e42838aae54a6a8871607f6399aecf. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
subset = dataset[datatype]['facts'][low_base:high_base]

In [ ]:
len(subset)

200

In [ ]:
import pandas as pd
import re
import nltk

In [ ]:
if True:
    batch_size = 4
    for case_id, case in enumerate(tqdm(subset, desc='Cases:')):
        parsed_case = []
        for string in tqdm(case[-64:]):
            parsed_facts = []
            string = nltk.sent_tokenize(string)
            string_arr = [s for s in string if len(s)>2]
            if True:
                for idx, string in enumerate([string_arr[i:i+batch_size] for i in range(0, batch_size, len(string_arr))]):
                    batch = [(idx*batch_size+idx_i, string_i, len(string_i.split())) for idx_i, string_i in enumerate(string)]
                    if not batch:
                        parsed_facts.append("")
                        continue
                    ids, sentences, _ = zip(*batch)
                    x, _ = tokenizer.batch_encode_sentences(sentences, device=device)
                    with torch.no_grad():
                        model.amr_mode = True
                        out = model.generate(**x, max_length=512, decoder_start_token_id=0, num_beams=args['beam_size'])
                    bgraphs = []
                    for idx, sent, tokk in zip(ids, sentences, out):
                        graph, status, (lin, backr) = tokenizer.decode_amr(tokk.tolist(), restore_name_ops=args['restore_name_ops'])
                        if args['only_ok'] and ('OK' not in str(status)):
                            continue
                        graph.metadata['status'] = str(status)
                        graph.metadata['source'] = 'path'
                        graph.metadata['nsent'] = str(idx)
                        graph.metadata['snt'] = sent
                        bgraphs.append((idx, graph))
                    output = ""
                    for i, g in bgraphs:
                        out = encode(g)
                        output += out+"\n"
                    try:
                        output = output[output.index('# ::snt'):]
                        output = output[output.index('\n'):]
                    except:
                        pass
                    output = re.sub(r'^\s*#.*$', '', output, flags=re.MULTILINE)
                    output = re.sub(r'\n+', '\n\n', output)
                    parsed_facts.append(output)
            parsed_case.append("\n".join(parsed_facts))
        case_df = pd.DataFrame({"parsed": parsed_case})
        case_df.to_csv("Case_"+str(low_base+case_id)+".csv", index=False)

 82%|████████▏ | 40/49 [02:02<00:23,  2.58s/it]Building failure:
['(', '<pointer:0>', 'impose-01', ':polarity', '-', ':ARG0', '(', '<pointer:1>', 'state', ':quant', 2, ':ARG1-of', '(', '<pointer:2>', 'include-91', ':ARG2', '(', '<pointer:3>', 'state', ':quant', 32, ':ARG0-of', '(', '<pointer:4>', 'have-org-role-91', ':ARG2', '(', '<pointer:5>', 'member', ')', ')', ')', ')', ':example', '(', '<pointer:6>', 'and', ':op1', '(', '<pointer:7>', 'country', ':wiki', '"Armenia"', ':name', '(', '<pointer:8>', 'name', ':op1', '"Armenia"', ')', ')', ':op2', '(', '<pointer:9>', 'country', ':wiki', '"Azerbaijan"', ':name', '(', '<pointer:10>', 'name', ':op1', '"Azerbaijan"', ')', ')', ':op3', '(', '<pointer:11>', 'country', ':wiki', '"Bosnia_and_Herzegovina"', ':name', '(', '<pointer:12>', 'name', ':op1', '"Bosnia"', ':op2', '"and"', ':op3', '"Herzegovina"', ')', ')', ':op4', '(', '<pointer:13>', 'country', ':wiki', '"Croatia"', ':name', '(', '<pointer:14>', 'name', ':op1', '"Croatia"', ')', ')', '

In [ ]:
!mkdir downloads

In [ ]:
import os
import shutil

for file in [f for f in os.listdir() if '.csv' in f]:
    shutil.move(file, "./downloads/"+file)

In [ ]:
!zip -r downloads.zip ./downloads

  adding: downloads/ (stored 0%)
  adding: downloads/Case_234.csv (deflated 87%)
  adding: downloads/Case_220.csv (deflated 86%)
  adding: downloads/Case_381.csv (deflated 89%)
  adding: downloads/Case_326.csv (deflated 87%)
  adding: downloads/Case_262.csv (deflated 86%)
  adding: downloads/Case_200.csv (deflated 85%)
  adding: downloads/Case_237.csv (deflated 83%)
  adding: downloads/Case_347.csv (deflated 88%)
  adding: downloads/Case_341.csv (deflated 87%)
  adding: downloads/Case_209.csv (deflated 87%)
  adding: downloads/Case_225.csv (deflated 87%)
  adding: downloads/Case_283.csv (deflated 83%)
  adding: downloads/Case_273.csv (deflated 87%)
  adding: downloads/Case_332.csv (deflated 88%)
  adding: downloads/Case_336.csv (deflated 88%)
  adding: downloads/Case_387.csv (deflated 88%)
  adding: downloads/Case_226.csv (deflated 86%)
  adding: downloads/Case_298.csv (deflated 89%)
  adding: downloads/Case_223.csv (deflated 86%)
  adding: downloads/Case_258.csv (deflated 86%)
  addin